In [9]:
import numpy as np
import pandas as pd


In [10]:
df = pd.read_csv('diabetes.csv')

In [11]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [13]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [14]:
!pip install scikit-learn

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [16]:
X = scaler.fit_transform(X)

In [17]:
X.shape

(768, 8)

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=1)

In [19]:
import tensorflow

In [75]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [21]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(X_train,y_train, batch_size=32, epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 2s 18ms/step - loss: 0.7665 - accuracy: 0.4739 - val_loss: 0.7227 - val_accuracy: 0.5390
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6938 - accuracy: 0.6107 - val_loss: 0.6646 - val_accuracy: 0.6623
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6431 - accuracy: 0.6873 - val_loss: 0.6198 - val_accuracy: 0.6753
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6067 - accuracy: 0.6857 - val_loss: 0.5877 - val_accuracy: 0.6883
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5794 - accuracy: 0.6971 - val_loss: 0.5625 - val_accuracy: 0.7143
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5571 - accuracy: 0.6954 - val_loss: 0.5434 - val_accuracy: 0.7662
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5395 - accuracy: 0.7101 - val_loss: 0.5286 - val_accuracy: 0.7662
Epoch 8/100


In [23]:
# 1. how to select appropriate optimizer
# 2. No, of nodes in a layer
# 3. how to select no. of layers
# 4. All in all one model

In [24]:
pip install -U keras-tuner

In [25]:
import kerastuner as kt

In [26]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values = ['adam','sdg','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [27]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [28]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 07s


In [29]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [30]:
model = tuner.get_best_models(num_models=1)[0]

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 18ms/step - loss: 0.5332 - accuracy: 0.7427 - val_loss: 0.5156 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 10ms/step - loss: 0.5183 - accuracy: 0.7524 - val_loss: 0.5027 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5062 - accuracy: 0.7638 - val_loss: 0.4946 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4969 - accuracy: 0.7704 - val_loss: 0.4868 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4895 - accuracy: 0.7720 - val_loss: 0.4804 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4827 - accuracy: 0.7801 - val_loss: 0.4752 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4772 - accuracy: 0.7818 - val_loss: 0.4721 - val_accuracy: 0.7922
Epoch 1

In [33]:
#to find no of neuron

In [49]:
def build_model(hp):

  model = Sequential()

  #units = hp.Int('units',min_value = 8,max_value = 128,step=8)
  units = hp.Int('units',min_value = 8,max_value = 128)

  model.add(Dense(units=units, activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [50]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='KerasDemo')

Reloading Tuner from mydir/KerasDemo/tuner0.json


In [51]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [52]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [53]:
model = tuner.get_best_models(num_models=1)[0]

In [54]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 2ms/step - loss: 0.5115 - accuracy: 0.7785
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4898 - accuracy: 0.7834
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4761 - accuracy: 0.7883
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4675 - accuracy: 0.7899
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4615 - accuracy: 0.7785
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4569 - accuracy: 0.7883
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4541 - accuracy: 0.7866
Epoch 14/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4513 - accuracy: 0.7769
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4493 - accuracy: 0.7801
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4473 - accuracy: 0.7850
Epo

In [48]:
# how to select no of layers

In [55]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(128,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [57]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='num_layers')

In [58]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 08s


In [59]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [60]:
model = tuner.get_best_models(num_models=1)[0]

In [61]:
model.fit(X_train,y_train,epochs=100,initial_epoch=7,validation_data=(X_test,y_test))

Epoch 8/100
20/20 [==============================] - 2s 17ms/step - loss: 0.4506 - accuracy: 0.7818 - val_loss: 0.4995 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4436 - accuracy: 0.7866 - val_loss: 0.5181 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4388 - accuracy: 0.7850 - val_loss: 0.4853 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4272 - accuracy: 0.8013 - val_loss: 0.5598 - val_accuracy: 0.7468
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4067 - accuracy: 0.8143 - val_loss: 0.5420 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4025 - accuracy: 0.8290 - val_loss: 0.5017 - val_accuracy: 0.7857
Epoch 14/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3800 - accuracy: 0.8176 - val_loss: 0.4795 - val_accuracy: 0.8052
Epoch 1

In [62]:
#Accuracy is not good hence making following changes

In [76]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:

      model.add(Dense(hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),activation = hp.Choice('activation' + str(i), values=['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),activation = hp.Choice('activation' + str(i), values=['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [77]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory="mydir",
                        project_name='final1')

In [80]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))


In [81]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 96,
 'activation0': 'relu',
 'dropout0': 0.5,
 'optimizer': 'sgd'}

In [82]:
model = tuner.get_best_models(num_models=1)[0]

In [83]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 [==============================] - 1s 13ms/step - loss: 0.6050 - accuracy: 0.6987 - val_loss: 0.5811 - val_accuracy: 0.7662
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5913 - accuracy: 0.7231 - val_loss: 0.5703 - val_accuracy: 0.7792
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6010 - accuracy: 0.6987 - val_loss: 0.5607 - val_accuracy: 0.7792
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5785 - accuracy: 0.7101 - val_loss: 0.5520 - val_accuracy: 0.7792
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5750 - accuracy: 0.6938 - val_loss: 0.5453 - val_accuracy: 0.7857
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5546 - accuracy: 0.7459 - val_loss: 0.5393 - val_accuracy: 0.7922
Epoch 13/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5572 - accuracy: 0.7345 - val_loss: 0.5334 - val_accuracy: 0.7922
Epoch 14